In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

Use Masking Layer to create an ImageNet that only predicts for Stanford Dogs dataset.
Need mapping list of indices between full ImageNet and Stanford Dogs.

Concept:
start with existing Stanford Dogs notebook about transfer learning.
Need mapping list of indices between full ImageNet and Stanford Dogs.

Load ImageNet model.
Download any subset of ImageNet with original indexes, preferably a mix with some dogs.
Predict from subset
Display misclassified image pairs. 


Stanford Dogs section
Create 2-way mapping tables
Predict SD only:
Create Dataset which maps index to ImageNet index
Predict Stanford Dogs from ImageNet with evaluate.
Predict again, pulling logits, counting misclassified outside of SD index list.
Display misclassified outside 

Save weights to file.
Pop final Dense layer.
Get name and config params, verify activation = softmax
Create new Dense layer with same shape and name, no activation.
Create simple Model with just Dense layer
'load weights by name' into new Model
now have Dense with no activation
Add Dense to ImageNet model.
Add Masking to ImageNet model.
Add Softmax layer.



In [2]:

IMG_SIZE = 224
DENSE_INPUT = 1280
IMAGENET_CLASSES = 1000
STANFORD_DOGS_CLASSES = 120
batch_size = 64



The Stanford Dogs image set is a subset of ImageNet-1000, and occupies a few ranges of indexes in the larger dataset. (Stanford Dogs does not include several wild canid species, sticking entirely to domestic breeds.) The Stanford Dogs label range is different than IN-1k, so we have to map the indexes back to the IN-1k index range. 

In [3]:
def sd2imagenet(index):
    if index >= 0 and index <= 99:
        return index + 151
    elif index >= 100 and index <= 116:
        return index + 152
    elif index >= 117 and index <= 119:
        return index + 156
    else:
        return -1

def imagenet2sd(index):
    if index >= 151 and index <= 250:
        return index - 151
    elif index >= 252 and index <= 268:
        return index - 152
    elif index >= 273 and index <= 275:
        return index - 156
    else:
        return -1

def get_sd_mask():
    masks = [0.0] * IMAGENET_CLASSES 
    for index in range(STANFORD_DOGS_CLASSES):
        masks[sd2imagenet(index)] = 1.0
    return masks

STANFORD_DOGS_MASK = get_sd_mask()
ANIMALS_MASK = np.zeros((IMAGENET_CLASSES,))
ANIMALS_MASK[:398] = 1.0
ANIMALS_MASK = ANIMALS_MASK.tolist()

Load the Stanford Dogs TF Dataset, prepare iterators, and map the indexes.

In [4]:
dataset_name = "stanford_dogs"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
def input_preprocess(image, label):
    mapped = sd2imagenet(int(label))
    onehot = tf.one_hot(mapped, IMAGENET_CLASSES)
    return image, onehot
ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))
# forget input augmentation as we are not training
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)


In [6]:
def build_masked_effnet(mask):
    assert type(mask) == type([]) and type(mask[0]) == type(1.0)
    effnet = EfficientNetB0(weights='imagenet')
    effnet.trainable = False
    m = tf.keras.models.Sequential()
    m.add(effnet)

    mask_layer = tf.keras.layers.Lambda(lambda x: x * mask)
    m.add(mask_layer)
    softmax_layer = tf.keras.layers.Softmax()
    m.add(softmax_layer)
    m.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy"])
    m.trainable = False
    return m

In [7]:
effnet = EfficientNetB0(weights='imagenet')
effnet.trainable = False
effnet.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy"])
_, original_accuracy_train = effnet.evaluate(ds_train)
_, original_accuracy_test = effnet.evaluate(ds_test)

134/134 [==============================] - 15s 113ms/step - loss: 0.6182 - accuracy: 0.8631


In [8]:
animals_model = build_masked_effnet(ANIMALS_MASK)
_, animals_accuracy_train = animals_model.evaluate(ds_train)
_, animals_accuracy_test = animals_model.evaluate(ds_test)

134/134 [==============================] - 15s 112ms/step - loss: 6.2511 - accuracy: 0.8656


In [9]:
dogs_model = build_masked_effnet(STANFORD_DOGS_MASK)
_, dogs_accuracy_train = dogs_model.evaluate(ds_train)
_, dogs_accuracy_test = dogs_model.evaluate(ds_test)

134/134 [==============================] - 15s 114ms/step - loss: 6.2511 - accuracy: 0.8693


In [10]:
print('Accuracy for full 1k classes, training/testing\n\t', original_accuracy_train, '\n\t', original_accuracy_test)
print('Accuracy for class set of all animals, training/testing\n\t', animals_accuracy_train, '\n\t', animals_accuracy_test)
print('Accuracy for Stanford Dogs class set, training/testing\n\t', dogs_accuracy_train, '\n\t', dogs_accuracy_test)

Accuracy for full 1k classes, training/testing
	 0.850685179233551 
	 0.863106369972229
Accuracy for class set of all animals, training/testing
	 0.852439820766449 
	 0.8655550479888916
Accuracy for Stanford Dogs class set, training/testing
	 0.8556985259056091 
	 0.8692863583564758
